# Control flow

## Topics
- conditional statements
- repeated evaluation
    - `while` and `for`
- breaking and continuing 

See also the [documentation](https://docs.julialang.org/en/v1/manual/control-flow/
)

## Epidemic Simulation

We can extend almost any Julia function with custom behaviour for our custom datatypes.

Case in point, let's print the Cell type in a prettier way.

In [ ]:
# Copy necessary things from the previous session

"Enumerate possible states of a single cell"
@enum InfectionStatus uninfected infected dead recovered

"Data structure containing the infection status of a cell"
mutable struct Cell
    status::InfectionStatus
    infection_time::Int8
end

In [ ]:
# This is how we write a print function for a cell
function Base.show(io::IO, cell::Cell)
    if cell.status == infected
        print(io, "◼")
    else
        print(io, "◻")
    end
end

In [ ]:
[cell1, cell2]

The interaction function in the previous session is not exactly what we wanted. The cell should become infected with a given probability.
    
Here we use a common trick to simulate a probability: if a random number between 1 and 0 is smaller than the probability, the cell gets infected.

In [ ]:
"""Simulate an interaction between two cells. In the other cell is
   infected, it may infect the this cell.
"""
function interact!(this_cell::Cell, other_cell::Cell, infection_rate)
    if this_cell.status == uninfected && other_cell.status == infected
        if rand(1)[1] < infection_rate
            this_cell.status = infected
            this_cell.infection_time = 0
        end
    end
end

In [ ]:
# Create an infected and an uninfected cell
cell1 = Cell(uninfected, 0)
cell2 = Cell(infected, 0)

println(cell1)
interact!(cell1, cell2, 0.1)
println(cell1)

## Loops

More details on loops and some tips and tricks

In [ ]:
for n in 1:10
    println(n)
end

#### Iterating over a list
For loops can also be used to iterate over containers

In [ ]:
for i in [1,4,0]
    println(i)
end

In [ ]:
for s ∈ ["foo","bar","baz"]
    println(s)
end

#### Enumerate: A life saver
Iterate over an array with index AND value. 

Syntax is
```julia
for (index, value) in enumerate(arr)
    println((index,value))
end
```

In [ ]:
for (i,s) ∈ enumerate(["foo","bar","baz"])
    println(i, " ", s)
end

## Breaking and continuing
Sometimes you need to terminate a `while` or `for` evaluation before the end.  This can be accomplished with the `break` keyword


In [ ]:
i = 1;
while true
    println(i)
    if i >= 5
        break
    end
    i += 1
end

In [ ]:
for i = 1:1000
    println(i)
    if i >= 5
        break
    end
end

In other circumstances, it is handy to be able to stop an iteration and move on to the next one immediately. The `continue` keyword accomplishes this.


In [ ]:
for i = 1:10
    if i % 3 != 0
        continue
    end
    println(i)
end

## Nested loops
Multiple nested loops can be combined into a single outer loop. 

In [ ]:
for i = 1:2
    for j = 3:3
        println((i,j))
    end
end

Translates into:

In [ ]:
for i = 1:2, j = 3:4
    println((i, j))
end

A `break` statement inside such a loop exits the entire nest of loops, not just the inner one.


## Some higher-order functions: `map`

`map` is a "higher-order" function in Julia that takes a *function* as one of its input arguments. `map` then applies that function to every element of the data structure you pass. 

For example
```julia
map(f, [1,2,3])
```
will correspond to
```julia
[f(1), f(2), f(3)]
```

In [ ]:
function f(x, y)
    return x + y
end

map(f, [1,2,3], [4,5,6])

## Some higher-order functions: `broadcast`
`broadcast` is another higher-order function like `map`. `broadcast` is actually a generalization, so it can do what `map` but also more!

Syntax is the same
```julia
broadcast(f, [1,2,3])
```

And so we have again applied f (squared) to all elements of `[1,2,3]`.

In [ ]:
broadcast(f, [1,2,3], [4,5,6])

In [ ]:
broadcast(f, [[4, 5, 6], [1, 2, 3]], [[1,2,3]])

## Broadcasting (or vectorizing)
Some syntactic sugar for calling `broadcast` is to place `.` between the name of the function you want to broadcast and its input arguments. 
        
For example
```julia
broadcast(f, [1,2,3])
```
is the same as
```julia
f.([1,2,3])
```

Note that this is not the same as `f([1,2,3])` because we can not square a vector!

Let's try broadcasting for a matrix `A`

In [ ]:
function f(x)
    return x^2
end

A = [1 2 3;
     4 5 6;
     7 8 9]

In [ ]:
f(A)

In [ ]:
f.(A)

## Dot syntax for vectorization
The dot syntax allows to write complex compound **elementwise** expressions in a way that looks natural/closer to mathematical notation. 

For example:

In [ ]:
A + 2 .* f.(A) ./ A

Instead of the more nasty looking version with `broadcast` as

In [ ]:
broadcast(x-> x + 2 * f(x) / x, A) 

The `.`-syntax is useful and looks nice, if you are used to interpreted languages beware: in Julia a for loop is faster. You don't need to vectorize to get good performance.

## Back to the simulation

Now we have everything we need to advance the simulation. Firsts let's create a starting state, where one cell in the middle is infected.

In [ ]:
# Create a 16 x 16 matrix of cells
cells = Matrix{Cell}(undef, 16, 16)

for i in 1:size(cells)[1]
    for j in 1:size(cells)[2]
        cells[i,j] = Cell(uninfected, 0)
    end
end
cells[8,8].status = infected

Now we can implement a time step in our simulation. For this, we need to loop over each pair of neighbouring cells, since they can infect each other.

First let's check the neighbours in the vertical direction. So if one cell is at (i,j), the cell at (i+1,j) is a neighbour. If a column has N cells, there are N-1 pairs.

In [ ]:
"Update the simulation one time step"
function update!(cells)
    # Create a copy to remember the old state
    old_cells = deepcopy(cells)
    
    # Find the number of cells in each direction
    Nx = size(cells)[1]
    Ny = size(cells)[2]    
    
    # Loop over pairs of cells in the same row. There are Nx-1 pairs.
    for j in 1:Ny
        # loop over all columns
        for i in 1:Nx-1
            # So the cells are (i+1,j) and (i,j). Each will 
            # interact with the other.
            
            interact!(cells[i,j], old_cells[i+1,j], 0.1)
            interact!(cells[i+1,j], old_cells[i,j], 0.1)

        end
    end
end

Just showing the array works quite well already, but we can do better. Let's extend the show function again:

In [ ]:
cells

Now the disease can spread vertically. The horizontal direction is one of the exercises.

In [ ]:
update!(cells)
print(cells)

# Extra:

## While statement
Syntax is 
```julia
while *condition*
    *loop body*
end
```

In [ ]:
n = 0
while n < 10
    n += 1
    println(n)
end

## List comprehension
Comprehensions provide a general and powerful way to construct arrays. 

Comprehension syntax is similar to set construction notation in mathematics
```julia
A = [ F(x,y,...) for x=rx, y=ry, ... ]
```

In [ ]:
[(i,j) for i=1:2 for j=1:i]

## Ternary operator
Even though the name for this operation is scary, it is actually very easy to understand and handy to use. Syntax is:
```julia
*condition* ? *do 1* : *do 2*
```
which is equal to writing
```julia
if *condition*
    *do 1*
else
    *do 2*
end
```

In [ ]:
# What does the followning code do? Try it out by giving values to `x` and `y`
x = 
y = 
(x > y ) ? x : y

### Advanced: SIMD vectorization
Vectorization is discussed more in the bonus notebook about SIMD vectorization.

In short, the topic is quite technical but you should rest assured that the dot syntax actually works quite well to make your code easy to read **and** fast to run.